# Introduction:

If you're like me, you love quality Asian food, more specifically Chinese dumplings. So, if you are traveling to a new city you are eager to find the best places to chow down. The problem that I will be solving for, is leveraging Chinese restaurant densities within neighborhoods of some of the largest cities in North America to determine where someone from out of town should go in order to fill their bellies on yummy food.

# Data Solution Proposal:

I will use the FourSquare API to collect data about locations of Chinese restaurants in 5 major North American cities which are: New York, NY, San Francisco, CA, Atlanta, GA, Toronto, ON and Chicago, IL. Given that these are highly populated cities in North America, I am confident that we will be able to glean the best areas within each city to fulfill one's craving for Chinese food